# Imports and model functions

In [32]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, make_scorer

import time
import psutil
import threading
from memory_profiler import memory_usage

In [27]:
def apply_rf(X_train, y_train, best_params=None, random_state=42, n_jobs=-1, cv=5): 
    measurement_rf = {}
    best_params = best_params or {}

    rf_model = RandomForestClassifier(**best_params, random_state=random_state, n_jobs=n_jobs, verbose=1)
    
    cpu_usage = []
    stop_flag = threading.Event()

    def monitor_cpu():
        while not stop_flag.is_set():
            cpu_usage.append(psutil.cpu_percent(interval=0.1))

    def train_model():
        rf_model.fit(X_train, y_train)

    try:
        cpu_thread = threading.Thread(target=monitor_cpu)
        cpu_thread.start()

        start_time = time.time()
        train_model()
        training_time = time.time() - start_time

        stop_flag.set()
        cpu_thread.join()

        measurement_rf['Training Time (s)'] = training_time
        measurement_rf['Peak CPU Usage (%)'] = max(cpu_usage)
        measurement_rf['Average CPU Usage (%)'] = sum(cpu_usage) / len(cpu_usage) if cpu_usage else 0

        # Modified to use F1 score
        f1_scorer = make_scorer(f1_score, average='weighted')
        cv_scores_rf = cross_val_score(rf_model, X_train, y_train, cv=cv, n_jobs=n_jobs, scoring=f1_scorer)

        return cv_scores_rf, measurement_rf, rf_model

    except Exception as e:
        import traceback
        print("⛔ Full error traceback:")
        traceback.print_exc()
        print(f"Error during Random Forest training: {e}")
        return None, None, None

In [55]:
def eval_dataset_w_RF(X_train, X_test, y_train, y_test, params_rf={'n_estimators': 150, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10}):
    # Fitting the model
    cv_scores_rf, measurement_rf, rf_model = apply_rf(X_train, y_train, best_params=params_rf)

    # Making predictions
    y_pred_rf = rf_model.predict(X_test)
    
    # Evaluating the model performance
    cv_scores_mean_rf = np.mean(cv_scores_rf)
    print(f'Cross validation average score: {cv_scores_mean_rf:.4f} +/- standard deviation: {np.std(cv_scores_rf):.4f}')

    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    print(f'Accuracy on the test set: {accuracy_rf:.4f}')
    
    # Checking computational cost
    print("Resource measurements:", measurement_rf)
    print(classification_report(y_test, y_pred_rf, digits=4))

In [ ]:
import optuna
def show_results(X_train, X_test, y_train, y_test, n_trials=100):
    def objective(trial, X_train, y_train, cv=5):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_categorical('max_depth', [None] + list(range(5, 31))),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
        }
        
        cv_scores, _, model = apply_rf(X_train, y_train, best_params=params, cv=cv)
        if cv_scores is None:
            return 0
        return np.mean(cv_scores)
        
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=n_trials)
        best_params = study.best_params
    
    cv_scores_rf, measurement_rf, rf_model = apply_rf(X_train, y_train, best_params=best_params)
    
    if cv_scores_rf is None:
        print("Model training failed")
        return
    
    y_pred_rf = rf_model.predict(X_test)
    
    # Convert to numpy arrays and ensure same type
    y_test_array = np.array(y_test)
    y_pred_array = np.array(y_pred_rf)
    
    # Print unique values to debug
    print("\nUnique values in test set:", np.unique(y_test_array))
    print("Unique values in predictions:", np.unique(y_pred_array))
    
    cv_scores_mean_rf = np.mean(cv_scores_rf)
    
    try:
        f1 = f1_score(y_test_array, y_pred_array, average='weighted')
        accuracy = accuracy_score(y_test_array, y_pred_array)
        
        print("\nModel Evaluation Results:")
        print("-" * 50)
        print(f'Cross validation average score (F1): {cv_scores_mean_rf:.4f} +/- standard deviation: {np.std(cv_scores_rf):.4f}')
        print(f'F1 Score on test set: {f1:.4f}')
        print(f'Accuracy on test set: {accuracy:.4f}')
        print("\nResource Usage:")
        print("-" * 50)
        print("Resource measurements:", measurement_rf)
        print("\nDetailed Classification Report:")
        print("-" * 50)
        print(classification_report(y_test_array, y_pred_array))
    
    except Exception as e:
        print(f"Error during metric calculation: {str(e)}")
        print("Types in test set:", y_test_array.dtype)
        print("Types in predictions:", y_pred_array.dtype)
        raise
    
    return rf_model, best_params

In [30]:
def optimize_rf_hyperparameters(X_train, y_train, n_iter=100, cv=5, n_jobs=-1, random_state=42):
    param_dist = {
        'n_estimators': randint(50, 300),
        'max_depth': [None] + list(range(5, 31)),  # Changed to direct list
        'min_samples_split': randint(2, 20),
        'min_samples_leaf': randint(1, 10),
        'max_features': ['sqrt', 'log2', None]
    }
    
    base_rf = RandomForestClassifier(random_state=random_state)
    
    random_search = RandomizedSearchCV(
        estimator=base_rf,
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=cv,
        verbose=2,
        random_state=random_state,
        n_jobs=n_jobs,
        scoring='f1_weighted'  
    )
    
    random_search.fit(X_train, y_train)
    
    print("Best parameters:", random_search.best_params_)
    print("Best F1 score:", random_search.best_score_)
    
    return random_search.best_params_

# Prep for model training cicids2017

In [44]:
# Reading data
df = pd.read_csv("..\..\data prep\cicids2017_prep\cicids2017_42feat_97percent.csv")

In [48]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, BorderlineSMOTE

In [49]:
# Preparing training and test splits
X = df.drop('Attack Type', axis=1)
y = df['Attack Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [50]:
# Initialize scaling algos
MMS = MinMaxScaler()
X_train_MMS_scaled = MMS.fit_transform(X_train)
X_test_MMS_scaled = MMS.transform(X_test)

In [51]:
X_train_scaled_rus_MMS, y_train_scaled_rus_MMS = RandomUnderSampler(sampling_strategy={'Normal Traffic': 500000}, random_state=42).fit_resample(X_train_MMS_scaled, y_train)

In [52]:
X_train_resampled_scaled_MMS_SMOTE, y_train_resampled_scaled_MMS_SMOTE = SMOTE(sampling_strategy={'Bots': 7500, 'Web Attacks': 7500, 'Brute Force': 7000, 'Port Scanning': 70000, 'DDoS':90000, 'DoS': 200000}, random_state=42).fit_resample(X_train_scaled_rus_MMS, y_train_scaled_rus_MMS)

# Sync classes

In [19]:
# Function to combine classes
def combine_classes(y, class_mapping):
    return y.map(class_mapping)
# Define the mapping
class_mapping = {
    'Web Attacks': 'Other',
    'Port Scanning': 'Other',
    'Normal Traffic': 'Normal Traffic',
    'Bots': 'Bots',
    'Brute Force': 'Brute Force',
    'DDoS': 'DDoS',
    'DoS': 'DoS'
}

In [43]:
df["Attack Type"].unique()

array(['Normal Traffic', 'DDoS', 'Port Scanning', 'Bots', 'Web Attacks',
       'Brute Force', 'DoS'], dtype=object)

In [21]:
# Apply to all your sets
y_train = combine_classes(y_train, class_mapping)
y_test = combine_classes(y_test, class_mapping)

y_train_scaled_rus_MMS = combine_classes(y_train_scaled_rus_MMS, class_mapping)
y_train_resampled_scaled_MMS_SMOTE = combine_classes(y_train_resampled_scaled_MMS_SMOTE, class_mapping)

# Search best params for MMS SMOTE

In [ ]:
rf_model, best_params = show_results(X_train_resampled_scaled_MMS_SMOTE, 
                                    X_test_MMS_scaled,
                                    y_train_resampled_scaled_MMS_SMOTE, 
                                    y_test, 
                                    optimization_type='optuna',
                                    n_trials=25)

[I 2025-04-29 17:26:27,497] A new study created in memory with name: no-name-495c7c3a-774a-4f9c-ae7c-e462d85e15b2
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  3.2min finished
[I 2025-04-29 17:41:19,846] Trial 0 finished with value: 0.9892232682914923 and parameters: {'n_estimators': 80, 'max_depth': 25, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_features': None}. Best is trial 0 with value: 0.9892232682914923.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   50.5s
[Parallel(n_jobs=-1)]: Done 232 out of 232 | elapsed:  1.1min finished
[I 2025-04-29 17:46:58,327] Trial 1 finished with value: 0.9888474475978211 and parameters: {'n_estimators': 232, 'max_depth': 29, 'min_samples_sp

TypeError: '<' not supported between instances of 'str' and 'float'

In [57]:
eval_dataset_w_RF(X_train_resampled_scaled_MMS_SMOTE, X_test_MMS_scaled, y_train_resampled_scaled_MMS_SMOTE, y_test, params_rf={'n_estimators': 200, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None})

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:  7.2min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    2.3s
[Parallel(n_jobs=16)]: Done 200 out of 200 | elapsed:    2.7s finished


Cross validation average score: 0.9989 +/- standard deviation: 0.0001
Accuracy on the test set: 0.9988
Resource measurements: {'Training Time (s)': 433.61325693130493, 'Peak CPU Usage (%)': 100.0, 'Average CPU Usage (%)': 96.10674567000902}
                precision    recall  f1-score   support

          Bots     0.6744    0.9401    0.7854       584
   Brute Force     0.9946    0.9989    0.9967      2745
          DDoS     0.9995    0.9998    0.9996     38404
           DoS     0.9967    0.9993    0.9980     58124
Normal Traffic     0.9999    0.9987    0.9993    628518
 Port Scanning     0.9890    0.9993    0.9941     27208
   Web Attacks     0.9738    0.9844    0.9791       643

      accuracy                         0.9988    756226
     macro avg     0.9468    0.9887    0.9646    756226
  weighted avg     0.9989    0.9988    0.9988    756226



In [58]:
eval_dataset_w_RF(X_train_resampled_scaled_MMS_SMOTE, X_test_MMS_scaled, y_train_resampled_scaled_MMS_SMOTE, y_test, params_rf={'n_estimators': 115, 'max_depth': 30, 'min_samples_split': 11, 'min_samples_leaf': 1, 'max_features': None})

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 115 out of 115 | elapsed:  4.3min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 115 out of 115 | elapsed:    1.5s finished


Cross validation average score: 0.9988 +/- standard deviation: 0.0001
Accuracy on the test set: 0.9988
Resource measurements: {'Training Time (s)': 259.2027132511139, 'Peak CPU Usage (%)': 100.0, 'Average CPU Usage (%)': 91.41160512434057}
                precision    recall  f1-score   support

          Bots     0.6777    0.9435    0.7888       584
   Brute Force     0.9924    0.9989    0.9956      2745
          DDoS     0.9995    0.9998    0.9996     38404
           DoS     0.9968    0.9993    0.9980     58124
Normal Traffic     0.9998    0.9987    0.9993    628518
 Port Scanning     0.9890    0.9990    0.9940     27208
   Web Attacks     0.9664    0.9844    0.9753       643

      accuracy                         0.9988    756226
     macro avg     0.9460    0.9891    0.9644    756226
  weighted avg     0.9989    0.9988    0.9988    756226

